In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.graph_objs as go 
from plotly.offline import download_plotlyjs, plot, iplot
import plotly.express as px


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Code for gathering data can be found here: https://github.com/cpud/climb-plotly/blob/master/final2.ipynb

## It involves some webscraping and data cleaning/transformation. For this notebook, I just uploaded the data I've already gathered and cleaned into the 'us-climbing-data' dataset, as rock climbing data on MountainProject is not updated frequently and they explicitly ask for route data to be cached for at least a week. 

## While MountainProject already has a global map of climbing routes, I wanted to try gathering my own data and making my own vizualization. So, I scraped data about climbing routes in the US off of MountainProject and made a couple of choropleth maps. MountainProject's map, in my humble opinion, isn't necessarily as intuitive as it could be in regards to the distribution of climbing routes amongst states, so I aimed to make a map where it is immediately obvious and easier to see which states have more listed routes.  Their map can be found here: https://www.mountainproject.com/map .  Furthermore, I wanted to see the distriubtion of climbing routes by the type of climbing amongst states, so I made an animated map that allows one to select the type of climbing to be displayed in the map. 

## As one would expect, CA and CO dominate all the categories in terms of raw numbers. However, Wisconsin, New Hampshire, and Massachusetts all have a suprising number of boulders and trad climbs. 

## One thing to note is that many climbing areas and routes are secretive and won't be shared with the public, let alone posted on MountainProject, so these numbers aren't going to be perfect representations of the actual amount of climbing routes in any given state. 

#### ***Since this was originally made, some state pages have changed their layouts, making the 'number of crags' measuse displayed in the first map to be incorrect. For example, North Carolina's page used to just list out each crag on the page, but now the crags are separated by 4 different regions, where the actual list of areas for that region is found by clicking on the link to those regions. 

In [ ]:
agg_df = pd.read_csv('/kaggle/input/us-climbing-data/aggregate.csv')
by_type_df = pd.read_csv('/kaggle/input/us-climbing-data/byState.csv')

In [ ]:
# Aggregate number of climbs by state and type
by_type_df = by_type_df.groupby(by = ['ST','type']).sum()
# Set a colmn to the index so we can get repeating labels
by_type_df['STtype'] = by_type_df.index
# Get repeating State label
by_type_df['State'] = by_type_df['STtype'].apply(lambda x: x[0])
# Get repeating Type label
by_type_df['Type'] = by_type_df['STtype'].apply(lambda x: x[1])
# Drop column
by_type_df.drop('STtype', axis = 1, inplace = True)
# Relabel columns for choropleth
by_type_df.columns = ['Climbs', 'State', 'Type']

In [ ]:
data = dict(type='choropleth',
            colorscale = 'inferno',
            locations = list(agg_df['ST']),
            z = agg_df['Climbs'],
            locationmode = 'USA-states',
            text = agg_df['text'],
            marker = dict(line = dict(color = 'rgb(255,255,255)',width = 2)),
            colorbar = {'title':"Number of Climbs"}
            ) 

layout = dict(title = 'Number of Climbs By State',
              geo = dict(scope='usa',
                         showlakes = True,
                         lakecolor = 'rgb(85,173,240)')
             )

In [ ]:
choromap = go.Figure(data = [data],layout = layout)
iplot(choromap)

In [ ]:
fig = px.choropleth(by_type_df, 
                    locations = 'State',
                    color="Climbs", 
                    animation_frame="Type",
                    color_continuous_scale="agsunset",
                    locationmode='USA-states',
                    scope="usa",
                    title='Climbs by State and Type',
                    #height=600
                   )

In [ ]:
fig.show()